# 1. 필요한 파일 불러오기

- '광양시 5개 동 격자 데이터.csv' 

: 광양시의 주 5개 동의 격자 데이터를 QGIS를 이용해 만들었다.

In [2]:
from geoband.API import *
import pandas as pd

GetCompasData('SBJ_2107_002', '3', '3.광양시_금호동_집적장별_재할용품통계.csv')
GetCompasData('SBJ_2009_001', '17', '17.광양시_건물분포도(연면적)_격자(100X100).geojson')

금호동_재할용품통계 = pd.read_csv('3.광양시_금호동_집적장별_재할용품통계.csv')
df_5dong = pd.read_csv('광양시_5개_동_격자_데이터.csv')

# 2. 데이터 정제하기

## 2-1. 금호동 데이터만 저장

In [10]:
goldho = df_5dong[df_5dong['emd_nm']=='금호동']

## 2-2. 상권 데이터 중 금호동만 저장
2-2-1. 상권 데이터에 위경도 변수를 이용해서 geometry 값을 만든 후
geojson 파일로 저장한 후

2-2-2. QGIS를 이용해 주 5동 별 격자별 상권 개수 변수를 만들어 다시 그 데이터(금호동_격자별_상권정보.csv)를 불러온다.

### 2-2-1

In [4]:
df= pd.read_excel('광양시_상권정보.xlsx')
gold2 = df[df['행정동명']=='금호동']

import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, Polygon, LineString

gold2['geometry'] = gold2.apply(lambda row : Point([row['경도'], row['위도']]), axis=1)
gold2 = gpd.GeoDataFrame(gold2, geometry='geometry')

gold2.to_file('gold2_loc.geojson', driver='GeoJSON')

/opt/app-root/lib/python3.6/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### 2-2-2. 상권 개수 변수 추가

In [6]:
square_gold = pd.read_csv('금호동_격자별_상권정보.csv')

In [8]:
square_gold = square_gold[square_gold['emd_nm']=='금호동']

In [9]:
square_gold

,emd_cd,emd_nm,gid,상권개수
1555,46230110,금호동,라라212603,0
1556,46230110,금호동,라라199601,0
1557,46230110,금호동,라라209607,0
1558,46230110,금호동,라라207605,0
1559,46230110,금호동,라라201601,0
...,...,...,...,...
3837,46230110,금호동,라라237581,0
3838,46230110,금호동,라라223572,0
3839,46230110,금호동,라라190581,0
3840,46230110,금호동,라라248553,0


In [11]:
금호동_상권정보_인구정보 = pd.merge(square_gold,goldho, on='gid')

In [12]:
금호동_상권정보_인구정보.columns

Index(['emd_cd_x', 'emd_nm_x', 'gid', '상권개수', 'emd_cd_y', 'emd_nm_y', 'val'], dtype='object')

In [14]:
금호동_상권정보_인구정보2 = 금호동_상권정보_인구정보[['gid','상권개수','val','emd_nm_x']]

In [15]:
금호동_상권정보_인구정보2

,gid,상권개수,val,emd_nm_x
0,라라212603,0,154.0,금호동
1,라라199601,0,138.0,금호동
2,라라209607,0,77.0,금호동
3,라라207605,0,9.0,금호동
4,라라201601,0,132.0,금호동
...,...,...,...,...
2282,라라237581,0,NaN,금호동
2283,라라223572,0,NaN,금호동
2284,라라190581,0,NaN,금호동
2285,라라248553,0,NaN,금호동


## 2-3. 건물 연면적 변수 추가
건물 연면적 na값 0으로 바꿔주기

In [17]:
df1 = gpd.read_file('17.광양시_건물분포도(연면적)_격자(100X100).geojson')

In [18]:
df1['val'].describe()

count     5426.000000
mean       570.149652
std       2152.521517
min          4.930000
25%         66.405000
50%         99.695000
75%        270.145000
max      93850.090000
Name: val, dtype: float64

In [20]:
연면적 = df1.fillna(0)

금호동_상권정보_인구정보2, 연면적 합치기 =  df2

In [21]:
df2 = pd.merge(금호동_상권정보_인구정보2, 연면적, on='gid')

In [23]:
df2.shape

(2287, 6)

In [24]:
df2.head()

,gid,상권개수,val_x,emd_nm_x,val_y,geometry
0,라라212603,0,154.0,금호동,1083.21,"MULTIPOLYGON (((127.73215 34.93730, 127.73215 ..."
1,라라199601,0,138.0,금호동,2261.98,"MULTIPOLYGON (((127.71791 34.93553, 127.71791 ..."
2,라라209607,0,77.0,금호동,1023.00,"MULTIPOLYGON (((127.72887 34.94092, 127.72887 ..."
3,라라207605,0,9.0,금호동,676.50,"MULTIPOLYGON (((127.72668 34.93912, 127.72668 ..."
4,라라201601,0,132.0,금호동,2135.38,"MULTIPOLYGON (((127.72010 34.93552, 127.72010 ..."


In [25]:
df3 = df2.rename(columns={'val_x':'인구정보','emd_nm_x':'행정동명','val_y':'연면적'})

In [26]:
df3

,gid,상권개수,인구정보,행정동명,연면적,geometry
0,라라212603,0,154.0,금호동,1083.21,"MULTIPOLYGON (((127.73215 34.93730, 127.73215 ..."
1,라라199601,0,138.0,금호동,2261.98,"MULTIPOLYGON (((127.71791 34.93553, 127.71791 ..."
2,라라209607,0,77.0,금호동,1023.00,"MULTIPOLYGON (((127.72887 34.94092, 127.72887 ..."
3,라라207605,0,9.0,금호동,676.50,"MULTIPOLYGON (((127.72668 34.93912, 127.72668 ..."
4,라라201601,0,132.0,금호동,2135.38,"MULTIPOLYGON (((127.72010 34.93552, 127.72010 ..."
...,...,...,...,...,...,...
2282,라라237581,0,NaN,금호동,0.00,"MULTIPOLYGON (((127.75946 34.91741, 127.75946 ..."
2283,라라223572,0,NaN,금호동,0.00,"MULTIPOLYGON (((127.74411 34.90933, 127.74411 ..."
2284,라라190581,0,NaN,금호동,0.00,"MULTIPOLYGON (((127.70801 34.91751, 127.70801 ..."
2285,라라248553,0,NaN,금호동,0.00,"MULTIPOLYGON (((127.77142 34.89214, 127.77142 ..."


## 2-4. 금호동 재활용품 쓰레기 배출량 변수 추가
2-4-1. QGIS를 이용해서 '3.광양시_금호동_집적장별_재할용품통계.csv'의 행 별 값 좌표를 직접 찍어 금호동 격자별 재활용품통계 geojson 파일('금호동_재활용품통계_포인트.geojson')을 만들어준다.

2-4-2. 격자별 재활용품 통계 변수 추가( na 값은 0으로 채워주기)

2-4-3. trash 단위를 kg -> g으로 바꿔주기 위해 1000곱해주기

### 2-4-2

In [4]:
import geopandas as gpd
mid2 = gpd.read_file('금호동_재활용품통계_포인트.geojson')
mid2 = mid2[['trash','gid']]

/opt/app-root/lib/python3.6/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [5]:
mid2 =pd.DataFrame(mid2.groupby('gid')['trash'].sum()).reset_index()

In [6]:
mid2

,gid,trash
0,라라197599,15.6
1,라라198597,18.2
2,라라198598,24.7
3,라라198599,16.0
4,라라198600,10.5
5,라라198601,27.3
6,라라199600,23.1
7,라라199604,1.5
8,라라199605,1.3
9,라라200597,5.9


In [69]:
latest2 = pd.merge(df3,mid2,on='gid',how='left')

In [70]:
latest2 = latest2.fillna(0)

In [71]:
latest2

,gid,상권개수,인구정보,행정동명,연면적,geometry,trash
0,라라212603,0,154.0,금호동,1083.21,"MULTIPOLYGON (((127.73215 34.93730, 127.73215 ...",12.9
1,라라199601,0,138.0,금호동,2261.98,"MULTIPOLYGON (((127.71791 34.93553, 127.71791 ...",0.0
2,라라209607,0,77.0,금호동,1023.00,"MULTIPOLYGON (((127.72887 34.94092, 127.72887 ...",0.0
3,라라207605,0,9.0,금호동,676.50,"MULTIPOLYGON (((127.72668 34.93912, 127.72668 ...",4.0
4,라라201601,0,132.0,금호동,2135.38,"MULTIPOLYGON (((127.72010 34.93552, 127.72010 ...",0.0
...,...,...,...,...,...,...,...
2282,라라237581,0,0.0,금호동,0.00,"MULTIPOLYGON (((127.75946 34.91741, 127.75946 ...",0.0
2283,라라223572,0,0.0,금호동,0.00,"MULTIPOLYGON (((127.74411 34.90933, 127.74411 ...",0.0
2284,라라190581,0,0.0,금호동,0.00,"MULTIPOLYGON (((127.70801 34.91751, 127.70801 ...",0.0
2285,라라248553,0,0.0,금호동,0.00,"MULTIPOLYGON (((127.77142 34.89214, 127.77142 ...",0.0


### 2-4-3

In [72]:
latest2['trash'] = latest2['trash']*1000

## 2-5. 격자중심과 가까운 건물 평균좌표와의 거리 변수 추가
2-5-1. QGIS를 이용해 격자중심과 가장 가까운 건물 평균좌표와의 거리.geojson 파일을 만들어준다. ('격자중심과 가까운 건물 평균좌표와의 거리 데이터.geojson')

2-5-2. 데이터 합치기

In [73]:
mid = gpd.read_file('격자중심과_가까운_건물_평균좌표와의_거리_데이터.geojson')
mid = mid[['gid','HubDist']]
mid

,gid,HubDist
0,라라181596,11.355533
1,라라083724,21.133983
2,라라111704,14.070054
3,라라118868,63.135008
4,라라206706,29.295002
...,...,...
47058,라라148712,567.905608
47059,라라142799,952.436805
47060,라라173613,22.277545
47061,라라054632,45.607290


In [74]:
latest2 = pd.merge(latest2,mid,on='gid',how='left')

In [75]:
latest2

,gid,상권개수,인구정보,행정동명,연면적,geometry,trash,HubDist
0,라라212603,0,154.0,금호동,1083.21,"MULTIPOLYGON (((127.73215 34.93730, 127.73215 ...",12900.0,13.308311
1,라라199601,0,138.0,금호동,2261.98,"MULTIPOLYGON (((127.71791 34.93553, 127.71791 ...",0.0,30.667224
2,라라209607,0,77.0,금호동,1023.00,"MULTIPOLYGON (((127.72887 34.94092, 127.72887 ...",0.0,36.987092
3,라라207605,0,9.0,금호동,676.50,"MULTIPOLYGON (((127.72668 34.93912, 127.72668 ...",4000.0,9.244004
4,라라201601,0,132.0,금호동,2135.38,"MULTIPOLYGON (((127.72010 34.93552, 127.72010 ...",0.0,17.696178
...,...,...,...,...,...,...,...,...
2282,라라237581,0,0.0,금호동,0.00,"MULTIPOLYGON (((127.75946 34.91741, 127.75946 ...",0.0,404.705986
2283,라라223572,0,0.0,금호동,0.00,"MULTIPOLYGON (((127.74411 34.90933, 127.74411 ...",0.0,217.809801
2284,라라190581,0,0.0,금호동,0.00,"MULTIPOLYGON (((127.70801 34.91751, 127.70801 ...",0.0,116.939443
2285,라라248553,0,0.0,금호동,0.00,"MULTIPOLYGON (((127.77142 34.89214, 127.77142 ...",0.0,214.805182


## 2-6. 격자중심과 가까운 버스정류장 거리 변수 추가
2-6-1. QGIS를 이용해 격자중심과 가장 가까운 버스정류장과의 거리.geojson 파일을 만들어준다. ('광양시_중심점별_가장_가까운_정류장_거리.geojson')

2-6-2. 데이터 합치기

In [76]:
bus = gpd.read_file('광양시_중심점별_가장_가까운_정류장_거리.geojson')
bus = bus.rename(columns={'HubDist' : 'Busdistance'})

In [77]:
latest2 = pd.merge(latest2,bus,on='gid',how='left')

In [78]:
latest2 = latest2.iloc[:,[0,1,2,4,5,6,7,11]]
latest2 = latest2.rename(columns={'geometry_x' : 'geometry'})
latest2

,gid,상권개수,인구정보,연면적,geometry,trash,HubDist,Busdistance
0,라라212603,0,154.0,1083.21,"MULTIPOLYGON (((127.73215 34.93730, 127.73215 ...",12900.0,13.308311,144.183573
1,라라199601,0,138.0,2261.98,"MULTIPOLYGON (((127.71791 34.93553, 127.71791 ...",0.0,30.667224,61.306397
2,라라209607,0,77.0,1023.00,"MULTIPOLYGON (((127.72887 34.94092, 127.72887 ...",0.0,36.987092,115.206453
3,라라207605,0,9.0,676.50,"MULTIPOLYGON (((127.72668 34.93912, 127.72668 ...",4000.0,9.244004,163.932305
4,라라201601,0,132.0,2135.38,"MULTIPOLYGON (((127.72010 34.93552, 127.72010 ...",0.0,17.696178,215.299204
...,...,...,...,...,...,...,...,...
2282,라라237581,0,0.0,0.00,"MULTIPOLYGON (((127.75946 34.91741, 127.75946 ...",0.0,404.705986,2019.788254
2283,라라223572,0,0.0,0.00,"MULTIPOLYGON (((127.74411 34.90933, 127.74411 ...",0.0,217.809801,2451.218420
2284,라라190581,0,0.0,0.00,"MULTIPOLYGON (((127.70801 34.91751, 127.70801 ...",0.0,116.939443,1789.629515
2285,라라248553,0,0.0,0.00,"MULTIPOLYGON (((127.77142 34.89214, 127.77142 ...",0.0,214.805182,4788.218514


# 3. 최종 데이터 프레임 만들기

In [79]:
latest2 = latest2[['gid','상권개수','인구정보','연면적','Busdistance','HubDist','trash','geometry']]

In [80]:
latest2

,gid,상권개수,인구정보,연면적,Busdistance,HubDist,trash,geometry
0,라라212603,0,154.0,1083.21,144.183573,13.308311,12900.0,"MULTIPOLYGON (((127.73215 34.93730, 127.73215 ..."
1,라라199601,0,138.0,2261.98,61.306397,30.667224,0.0,"MULTIPOLYGON (((127.71791 34.93553, 127.71791 ..."
2,라라209607,0,77.0,1023.00,115.206453,36.987092,0.0,"MULTIPOLYGON (((127.72887 34.94092, 127.72887 ..."
3,라라207605,0,9.0,676.50,163.932305,9.244004,4000.0,"MULTIPOLYGON (((127.72668 34.93912, 127.72668 ..."
4,라라201601,0,132.0,2135.38,215.299204,17.696178,0.0,"MULTIPOLYGON (((127.72010 34.93552, 127.72010 ..."
...,...,...,...,...,...,...,...,...
2282,라라237581,0,0.0,0.00,2019.788254,404.705986,0.0,"MULTIPOLYGON (((127.75946 34.91741, 127.75946 ..."
2283,라라223572,0,0.0,0.00,2451.218420,217.809801,0.0,"MULTIPOLYGON (((127.74411 34.90933, 127.74411 ..."
2284,라라190581,0,0.0,0.00,1789.629515,116.939443,0.0,"MULTIPOLYGON (((127.70801 34.91751, 127.70801 ..."
2285,라라248553,0,0.0,0.00,4788.218514,214.805182,0.0,"MULTIPOLYGON (((127.77142 34.89214, 127.77142 ..."


In [81]:
latest2.to_csv("금호동_최종_데이터.csv")